# Projeto utilizando Método das Árvores

### Instalação dos componentes relacionado ao Spark

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
import os
import findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"
findspark.init('spark-2.4.4-bin-hadoop2.7')

## Importação das Bibliotecas

In [3]:
import pyspark
from pyspark.sql import SparkSession

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.classification import DecisionTreeClassifier, GBTClassifier, RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

## Criando uma sessão e lendo a base




In [4]:
spark = SparkSession.builder.appName('tree_code').getOrCreate()

In [5]:
PATH = '/content/drive/MyDrive/Colab Notebooks/Estudos/Spark/Tree_Methods/base/College.csv'
data = spark.read.csv(PATH, inferSchema = True, header = True)
data.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [6]:
data.show(10)

+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+
|              School|Private|Apps|Accept|Enroll|Top10perc|Top25perc|F_Undergrad|P_Undergrad|Outstate|Room_Board|Books|Personal|PhD|Terminal|S_F_Ratio|perc_alumni|Expend|Grad_Rate|
+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+
|Abilene Christian...|    Yes|1660|  1232|   721|       23|       52|       2885|        537|    7440|      3300|  450|    2200| 70|      78|     18.1|         12|  7041|       60|
|  Adelphi University|    Yes|2186|  1924|   512|       16|       29|       2683|       1227|   12280|      6450|  750|    1500| 29|      30|     12.2|         16| 10527|       56|
|      Adrian College|    Yes|1428|  1097|   336|       22|       50|       1036|         99|  

In [7]:
data.head()

Row(School='Abilene Christian University', Private='Yes', Apps=1660, Accept=1232, Enroll=721, Top10perc=23, Top25perc=52, F_Undergrad=2885, P_Undergrad=537, Outstate=7440, Room_Board=3300, Books=450, Personal=2200, PhD=70, Terminal=78, S_F_Ratio=18.1, perc_alumni=12, Expend=7041, Grad_Rate=60)

In [8]:
data.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

# Transformando os dados

## Vetorizando os dados

In [9]:
inputCols = ['Apps',
            'Accept',
            'Enroll',
            'Top10perc',
            'Top25perc',
            'F_Undergrad',
            'P_Undergrad',
            'Outstate',
            'Room_Board',
            'Books',
            'Personal',
            'PhD',
            'Terminal',
            'S_F_Ratio',
            'perc_alumni',
            'Expend',
            'Grad_Rate']
assembler = VectorAssembler(inputCols = inputCols, outputCol = 'features')

In [10]:
output = assembler.transform(data)

## Tratando a coluna 'Private'

In [11]:
indexer = StringIndexer(inputCol = 'Private', outputCol = 'PrivateIndex')
output_fixed = indexer.fit(output).transform(output)

In [12]:
final_data = output_fixed.select('features', 'PrivateIndex')

## Separando os dados e criando os classificadores

In [13]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [14]:
dtc = DecisionTreeClassifier(labelCol = 'PrivateIndex', featuresCol = 'features')
rfc = RandomForestClassifier(labelCol = 'PrivateIndex', featuresCol = 'features')
gbt = GBTClassifier(labelCol = 'PrivateIndex', featuresCol = 'features')

In [15]:
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

## Comparação os modelos

In [16]:
dtc_predictions = dtc_model.transform(test_data)
rfc_predictions = rfc_model.transform(test_data)
gbt_predictions = gbt_model.transform(test_data)

In [17]:
acc_evaluator = MulticlassClassificationEvaluator(labelCol = 'PrivateIndex',
                                                  predictionCol = 'prediction',
                                                  metricName = 'accuracy')

In [18]:
dtc_acc = acc_evaluator.evaluate(dtc_predictions)
rfc_acc = acc_evaluator.evaluate(rfc_predictions)
gbt_acc = acc_evaluator.evaluate(gbt_predictions)

In [19]:
print('Here are the results!')
print('-' * 70)
print('A single decision tree had an accuracy of: {}'.format(dtc_acc))
print('-' * 70)
print('A random forest ensemble had an accuracy of: {}'.format(rfc_acc))
print('-' * 70)
print('A ensemble using GBT had an accuracy of: {}'.format(gbt_acc))
print('-' * 70)

Here are the results!
----------------------------------------------------------------------
A single decision tree had an accuracy of: 0.8765432098765432
----------------------------------------------------------------------
A random forest ensemble had an accuracy of: 0.9218106995884774
----------------------------------------------------------------------
A ensemble using GBT had an accuracy of: 0.8847736625514403
----------------------------------------------------------------------
